# 📘 Sesión 12: Proyecto Final Integrador

---

## 🎯 Objetivos

- Aplicar todos los conceptos del curso
- Realizar análisis exploratorio completo
- Crear visualizaciones profesionales
- Implementar modelo de ML end-to-end
- Documentar y presentar resultados

---

## 📋 Proyecto: Predicción de Precios de Viviendas

En este proyecto integrador, trabajaremos con un dataset de viviendas para:
1. Explorar y limpiar los datos
2. Visualizar patrones y relaciones
3. Preprocesar para ML
4. Entrenar y evaluar modelos
5. Interpretar resultados

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

sns.set_theme(style='whitegrid')
%matplotlib inline

## 1. Carga y Exploración de Datos

In [ ]:
# Crear dataset sintético de viviendas
np.random.seed(42)
n = 500

df = pd.DataFrame({
    'metros_cuadrados': np.random.randint(40, 300, n),
    'habitaciones': np.random.randint(1, 6, n),
    'baños': np.random.randint(1, 4, n),
    'antigüedad': np.random.randint(0, 50, n),
    'zona': np.random.choice(['Centro', 'Norte', 'Sur', 'Este', 'Oeste'], n),
    'tiene_garaje': np.random.choice([0, 1], n),
    'tiene_piscina': np.random.choice([0, 1], n, p=[0.8, 0.2]),
    'planta': np.random.randint(1, 10, n)
})

# Precio basado en features + ruido
df['precio'] = (
    df['metros_cuadrados'] * 2500 +
    df['habitaciones'] * 15000 +
    df['baños'] * 10000 -
    df['antigüedad'] * 1000 +
    df['tiene_garaje'] * 20000 +
    df['tiene_piscina'] * 30000 +
    df['zona'].map({'Centro': 50000, 'Norte': 30000, 'Sur': 20000, 'Este': 25000, 'Oeste': 15000}) +
    np.random.normal(0, 20000, n)
)

df['precio'] = df['precio'].clip(lower=50000)  # Precio mínimo

print("Primeras filas:")
print(df.head())

In [ ]:
# Información del dataset
print("\n📊 Información del Dataset:")
print(f"Dimensiones: {df.shape}")
print(f"\nTipos de datos:")
print(df.dtypes)
print(f"\nValores nulos:")
print(df.isnull().sum())

In [ ]:
# Estadísticas descriptivas
print("📈 Estadísticas:")
print(df.describe().round(2))

## 2. Análisis Exploratorio (EDA)

In [ ]:
# Distribución del precio (target)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(df['precio'], kde=True, ax=axes[0])
axes[0].set_title('Distribución del Precio')
axes[0].set_xlabel('Precio (€)')

sns.boxplot(data=df, x='zona', y='precio', ax=axes[1])
axes[1].set_title('Precio por Zona')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de correlación
fig, ax = plt.subplots(figsize=(10, 8))

numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, ax=ax)
ax.set_title('Matriz de Correlación')
plt.show()

In [ ]:
# Relaciones con el precio
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.scatterplot(data=df, x='metros_cuadrados', y='precio', hue='zona', alpha=0.6, ax=axes[0,0])
axes[0,0].set_title('Precio vs Metros Cuadrados')

sns.boxplot(data=df, x='habitaciones', y='precio', ax=axes[0,1])
axes[0,1].set_title('Precio por Habitaciones')

sns.scatterplot(data=df, x='antigüedad', y='precio', alpha=0.5, ax=axes[1,0])
axes[1,0].set_title('Precio vs Antigüedad')

sns.boxplot(data=df, x='tiene_garaje', y='precio', ax=axes[1,1])
axes[1,1].set_title('Precio con/sin Garaje')
axes[1,1].set_xticklabels(['Sin Garaje', 'Con Garaje'])

plt.tight_layout()
plt.show()

## 3. Preprocesamiento

In [ ]:
# Preparar features
df_model = df.copy()

# One-hot encoding para zona
df_model = pd.get_dummies(df_model, columns=['zona'], prefix='zona')

# Separar features y target
X = df_model.drop('precio', axis=1)
y = df_model['precio']

print(f"Features: {X.shape}")
print(f"Columnas: {X.columns.tolist()}")

In [ ]:
# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

## 4. Modelado

In [ ]:
# Comparar múltiples modelos
modelos = {
    'Linear Regression': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'Lasso': Lasso(alpha=1.0),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
}

resultados = []

for nombre, modelo in modelos.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', modelo)
    ])
    
    # Cross-validation
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='r2')
    
    # Entrenar y evaluar en test
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    resultados.append({
        'Modelo': nombre,
        'CV R² (mean)': cv_scores.mean(),
        'CV R² (std)': cv_scores.std(),
        'Test RMSE': rmse,
        'Test MAE': mae,
        'Test R²': r2
    })
    
    print(f"{nombre}: R²={r2:.4f}, RMSE={rmse:,.0f}€")

df_resultados = pd.DataFrame(resultados)
print("\n📊 Resumen de Modelos:")
print(df_resultados.round(4))

In [ ]:
# Visualizar comparación
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# R² Score
sns.barplot(data=df_resultados, x='Modelo', y='Test R²', ax=axes[0], palette='viridis')
axes[0].set_title('R² Score por Modelo')
axes[0].tick_params(axis='x', rotation=45)

# RMSE
sns.barplot(data=df_resultados, x='Modelo', y='Test RMSE', ax=axes[1], palette='viridis')
axes[1].set_title('RMSE por Modelo (€)')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Análisis del Mejor Modelo

In [ ]:
# Entrenar mejor modelo (Gradient Boosting o el mejor según resultados)
best_model = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingRegressor(n_estimators=100, random_state=42))
])

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

In [ ]:
# Predicciones vs Reales
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter plot
axes[0].scatter(y_test, y_pred, alpha=0.5)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Precio Real (€)')
axes[0].set_ylabel('Precio Predicho (€)')
axes[0].set_title('Predicciones vs Valores Reales')

# Residuos
residuos = y_test - y_pred
axes[1].hist(residuos, bins=30, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--')
axes[1].set_xlabel('Residuo (€)')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title('Distribución de Residuos')

plt.tight_layout()
plt.show()

In [ ]:
# Feature Importance
feature_importance = best_model.named_steps['model'].feature_importances_
feature_names = X.columns

df_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(df_importance['Feature'], df_importance['Importance'])
plt.xlabel('Importancia')
plt.title('Importancia de Features - Gradient Boosting')
plt.show()

## 6. Conclusiones y Recomendaciones

### 📊 Hallazgos Principales:

1. **Distribución de Precios**: El precio promedio es X€ con una distribución...

2. **Variables más Importantes**:
   - Metros cuadrados (correlación más alta)
   - Zona (Centro tiene precios más altos)
   - Número de habitaciones

3. **Rendimiento del Modelo**:
   - El mejor modelo fue Gradient Boosting con R² de X
   - Error promedio de predicción: X€

### 🎯 Recomendaciones:

1. Recopilar más datos sobre características adicionales
2. Considerar variables externas (distancia a transporte, servicios)
3. Segmentar modelos por zona para mayor precisión

---
## 📝 Ejercicios Adicionales

In [ ]:
# Ejercicio 1: Implementar GridSearchCV para optimizar hiperparámetros
# Tu código aquí

In [ ]:
# Ejercicio 2: Crear función predict_price(caracteristicas)
# Tu código aquí

In [ ]:
# Ejercicio 3: Implementar el mismo modelo con PyTorch
# Tu código aquí

---
## 🎓 Resumen del Curso

### Sesiones Completadas:

1. ✅ Fundamentos de Estructura de Código
2. ✅ PEP8 y Código Profesional
3. ✅ Funciones Avanzadas y Recursividad
4. ✅ Generadores e Iteradores
5. ✅ NumPy
6. ✅ Pandas Básico
7. ✅ Pandas Avanzado
8. ✅ Matplotlib
9. ✅ Seaborn
10. ✅ Scikit-learn
11. ✅ PyTorch
12. ✅ Proyecto Final

### 🚀 Próximos Pasos:

- Practicar con datasets reales de Kaggle
- Profundizar en Deep Learning
- Explorar MLOps y deployment
- Contribuir a proyectos open source

**¡Felicidades por completar el curso!** 🎉